_Lambda School Data Science, Unit 2_
 
# Linear Models  Sprint Challenge: Predict home prices in Ames, Iowa 🏠💸

To demonstrate mastery on your Sprint Challenge, do all the required, numbered instructions in this notebook.

To earn a score of "3", also do all the stretch goals.

For your Sprint Challenge, you'll use historical housing data. There's a data dictionary at the bottom of the notebook. 

Run this code cell to load the dataset:





In [144]:
# Importing Libarys
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%matplotlib inline
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import category_encoders as ce
from sklearn.linear_model import LinearRegression

In [145]:
# Importing Data
URL = 'https://drive.google.com/uc?export=download&id=1522WlEW6HFss36roD_Cd9nybqSuiVcCK'
df = pd.read_csv(URL)
assert df.shape == (2904, 47)

In [146]:
# Taking a look at the Dataset
df.describe() 

,1st_Flr_SF,Bedroom_AbvGr,Bsmt_Full_Bath,Bsmt_Half_Bath,Full_Bath,Gr_Liv_Area,Half_Bath,Kitchen_AbvGr,Lot_Area,MS_SubClass,Mo_Sold,Overall_Cond,Overall_Qual,SalePrice,TotRms_AbvGrd,Year_Built,Year_Remod/Add,Yr_Sold
count,2904.000000,2904.000000,2904.000000,2904.000000,2904.000000,2904.000000,2904.000000,2904.000000,2904.000000,2904.000000,2904.000000,2904.000000,2904.000000,2904.000000,2904.000000,2904.000000,2904.000000,2904.000000
mean,1158.257231,2.854339,0.431474,0.061639,1.562328,1497.853650,0.376722,1.044421,10130.199725,57.474174,6.206956,5.567149,6.086777,180383.689738,6.441804,1971.109504,1984.112259,2007.792011
std,388.351648,0.827820,0.525073,0.246202,0.551427,504.591527,0.501416,0.214260,7869.928702,42.724950,2.712204,1.113736,1.410083,79802.823684,1.573130,30.227141,20.857813,1.318719
min,334.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,1300.000000,20.000000,1.000000,1.000000,1.000000,12789.000000,2.000000,1872.000000,1950.000000,2006.000000
25%,876.750000,2.000000,0.000000,0.000000,1.000000,1124.000000,0.000000,1.000000,7424.750000,20.000000,4.000000,5.000000,5.000000,129150.000000,5.000000,1953.000000,1965.000000,2007.000000
50%,1083.500000,3.000000,0.000000,0.000000,2.000000,1440.500000,0.000000,1.000000,9429.000000,50.000000,6.000000,5.000000,6.000000,160000.000000,6.000000,1973.000000,1993.000000,2008.000000
75%,1383.000000,3.000000,1.000000,0.000000,2.000000,1740.500000,1.000000,1.000000,11516.250000,70.000000,8.000000,6.000000,7.000000,213000.000000,7.000000,2000.000000,2004.000000,2009.000000
max,5095.000000,8.000000,3.000000,2.000000,4.000000,5642.000000,2.000000,3.000000,215245.000000,190.000000,12.000000,9.000000,10.000000,755000.000000,15.000000,2010.000000,2010.000000,2010.000000


## 1. Do train/test split
Train on houses sold in 2006-08. Test on houses sold in 2009-10.

In [147]:
# Spliting Data set in to train and test
train = df[df['Yr_Sold'] < 2009]
test = df[df['Yr_Sold'] >= 2009]

In [148]:
# Size of train
train.Yr_Sold.count()

1920

In [149]:
#Size of test
test.Yr_Sold.count()

984

## 2. Begin with baselines

What is the Mean Absolute Error and R^2 score for a mean baseline?

In [150]:
# Seting my base line
mean_base = train.SalePrice.mean()

print("MAE :",mean_absolute_error(test.SalePrice, [mean_base] * test.Yr_Sold.count()))
print( "R squared :",r2_score(test.SalePrice, [mean_base] * test.Yr_Sold.count()))

MAE : 58276.87133934621
R squared : -0.0017922985422960114


## 3. Arrange data into X features matrix and y target vector
Select at least one numeric feature and at least one categorical feature.


In [162]:
# Looking at columns for features variables
train.columns

Index(['1st_Flr_SF', 'Bedroom_AbvGr', 'Bldg_Type', 'Bsmt_Full_Bath',
       'Bsmt_Half_Bath', 'Central_Air', 'Condition_1', 'Condition_2',
       'Electrical', 'Exter_Cond', 'Exter_Qual', 'Exterior_1st',
       'Exterior_2nd', 'Foundation', 'Full_Bath', 'Functional', 'Gr_Liv_Area',
       'Half_Bath', 'Heating', 'Heating_QC', 'House_Style', 'Kitchen_AbvGr',
       'Kitchen_Qual', 'Land_Contour', 'Land_Slope', 'Lot_Area', 'Lot_Config',
       'Lot_Shape', 'MS_SubClass', 'MS_Zoning', 'Mas_Vnr_Type', 'Mo_Sold',
       'Neighborhood', 'Overall_Cond', 'Overall_Qual', 'Paved_Drive',
       'Roof_Matl', 'Roof_Style', 'SalePrice', 'Sale_Condition', 'Sale_Type',
       'Street', 'TotRms_AbvGrd', 'Utilities', 'Year_Built', 'Year_Remod/Add',
       'Yr_Sold'],
      dtype='object')

In [163]:
# Seting up features and target 

features = ['MS_SubClass', 'Overall_Cond', 'Half_Bath', 'Bedroom_AbvGr','Overall_Qual','Gr_Liv_Area','1st_Flr_SF','Bsmt_Full_Bath','Full_Bath','TotRms_AbvGrd','Year_Built','Kitchen_AbvGr', 'Year_Remod/Add','Lot_Area','Bedroom_AbvGr',]

target = 'SalePrice'

# Training x and y variables
xtrain = train[features]
ytrain = train[target]

# Testing x and y variable
xtest = test[features]
ytest = test[target]

## 4. Do one-hot encoding

Encode your categorical feature(s).

In [153]:
# Encoding training and testing 
encoder = ce.OneHotEncoder(use_cat_names=True)

xtrain = encoder.fit_transform(xtrain)
xtest = encoder.transform(xtest)

# Scaling Training and testing 
scaler = StandardScaler()

xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)


/home/gunther/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/gunther/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/gunther/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


## 5. Use scikit-learn for Linear Regression
Fit your model.

In [154]:
# Defining Linerear Regression model
model = LinearRegression()

# Fiting model to training data
model.fit(xtrain, ytrain)

# making testing prediction
ypred = model.predict(xtest)

## 6. Use regression metrics: MAE, R^2
What is the Mean Absolute Error and R^2 score for your model? Beat the mean baseline!

In [166]:
# Calculating mean absolute error and r score

print("Base line MAE :",mean_absolute_error(test.SalePrice, [mean_base] * test.Yr_Sold.count()))
print( "Base line R squared :",r2_score(test.SalePrice, [mean_base] * test.Yr_Sold.count()))

print("Linear Regression MAE :",mean_absolute_error(ytest,ypred))

print("Linear Regression R squared :", r2_score(ytest,ypred))
# We beat our base line. 


Base line MAE : 58276.87133934621
Base line R squared : -0.0017922985422960114
Linear Regression MAE : 21821.220463722027
Linear Regression R squared : 0.836375357061377


## 7. Get coefficients of a linear model
Print or plot the coefficients for the features in your model.

In [156]:
model.coef_

array([-7387.04641796,  4777.17868287,  -744.49809391, -2532.57028901,
       30765.85717037, 33413.81444827,  5685.66717006,  9181.20600661,
        1595.69475046,   460.97165427, 13097.05980626, -2613.06931439,
        2493.73953986,  3684.32311773, -2532.57028901])

## Stretch Goals
- Try at least 3 different feature combinations.
- Plot exploratory visualizations.
- Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.